In [20]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.nn.init as init


data_loc = '/Users/simon/Documents/DTU/9. semester/deep learning/data'

df_all = []

files = os.listdir(os.path.join(data_loc,'modified data'))
for file in sorted(files):
    path = os.path.join(data_loc,'modified data',file)
    name, ext = os.path.splitext(file)
    if ext != '.csv':
        continue
    df = pd.read_csv(path)
    df.name = name
    
    for col_name in df.columns:
        if col_name != 'Date_Time':
            df[col_name]=df[col_name].astype('float64')
        else:
            df['Date_Time'] = pd.to_datetime(df['Date_Time'])
    
    df_all.append(df)
    print(name)
   

Measurement 1
Measurement 2
Measurement 3
NWP 1
NWP 2
NWP 3


In [21]:
case=1
target = torch.Tensor(df_all[case-1]['Park Power [KW]'].values)
target_time = df_all[case-1]['Date_Time']
x = torch.Tensor(df_all[case+2].iloc[:,1:].values)
x_time = df_all[case+2]['Date_Time']

In [57]:
assert x_time.diff().min()==x_time.diff().max()
assert target_time.diff().min()==target_time.diff().max()
assert x_time.diff().min()==target_time.diff().min()
assert target_time.iloc[-1]==x_time.iloc[-1]

time_dif = (target_time.iloc[0]-x_time.iloc[0])
idx_offset = time_dif.days*24*4+round(time_dif.seconds/(60*15))

idx_train_len = 24*4

i=1000
print(target_time[i])
print(x_time[i+idx_offset-idx_train_len+1:i+idx_offset+1])

2017-01-18 06:15:00
36618   2017-01-17 06:30:00
36619   2017-01-17 06:45:00
36620   2017-01-17 07:00:00
36621   2017-01-17 07:15:00
36622   2017-01-17 07:30:00
                ...        
36709   2017-01-18 05:15:00
36710   2017-01-18 05:30:00
36711   2017-01-18 05:45:00
36712   2017-01-18 06:00:00
36713   2017-01-18 06:15:00
Name: Date_Time, Length: 96, dtype: datetime64[ns]


In [3]:

# define network
class Net(nn.Module):

    def __init__(self, input_size, hidden_size, out_size):
        super(Net, self).__init__()  
        
        model = nn.Sequential(
            nn.LSTM(input_size=input_size,
                    hidden_size=hidden_size,
                    bias=True),
            nn.ReLu(),
            x = x.view(-1, hidden_size) # maybe use flatten?
            nn.Linear(in_features=hidden_size,
                      out_features=out_size,
                      bias=False)
        )


    def forward(self, x):
        x = self.model(x)
        return x


net = Net(input_size, hidden_size, out_size)
print(net)

optimizer = optim.Adam(net.parameters())
criterion = nn.MSELoss()

In [ ]:
# setting hyperparameters and gettings epoch sizes
batch_size = 100
num_epochs = 200
num_samples_train = x_train.shape[0]
num_batches_train = num_samples_train // batch_size
num_samples_valid = x_valid.shape[0]
num_batches_valid = num_samples_valid // batch_size

# setting up lists for handling loss/accuracy
train_acc, train_loss = [], []
valid_acc, valid_loss = [], []
test_acc, test_loss = [], []
cur_loss = 0
losses = []

get_slice = lambda i, size: range(i * size, (i + 1) * size)

for epoch in range(num_epochs):
    # Forward -> Backprob -> Update params
    ## Train
    cur_loss = 0
    net.train()
    for i in range(num_batches_train):
        optimizer.zero_grad()
        slce = get_slice(i, batch_size)
        output = net(x_train[slce])
        
        # compute gradients given loss
        target_batch = targets_train[slce]
        batch_loss = criterion(output, target_batch)
        batch_loss.backward()
        optimizer.step()
        
        cur_loss += batch_loss   
    losses.append(cur_loss / batch_size)

    net.eval()
    ### Evaluate training
    train_preds, train_targs = [], []
    for i in range(num_batches_train):
        slce = get_slice(i, batch_size)
        output = net(x_train[slce])
        
        preds = torch.max(output, 1)[1]
        
        train_targs += list(targets_train[slce].numpy())
        train_preds += list(preds.data.numpy())
    
    ### Evaluate validation
    val_preds, val_targs = [], []
    for i in range(num_batches_valid):
        slce = get_slice(i, batch_size)
        
        output = net(x_valid[slce])
        preds = torch.max(output, 1)[1]
        val_targs += list(targets_valid[slce].numpy())
        val_preds += list(preds.data.numpy())
        

    train_acc_cur = accuracy_score(train_targs, train_preds)
    valid_acc_cur = accuracy_score(val_targs, val_preds)
    
    train_acc.append(train_acc_cur)
    valid_acc.append(valid_acc_cur)
    
    if epoch % 10 == 0:
        print("Epoch %2i : Train Loss %f , Train acc %f, Valid acc %f" % (
                epoch+1, losses[-1], train_acc_cur, valid_acc_cur))

epoch = np.arange(len(train_acc))
plt.figure()
plt.plot(epoch, train_acc, 'r', epoch, valid_acc, 'b')
plt.legend(['Train Accucary','Validation Accuracy'])
plt.xlabel('Updates'), plt.ylabel('Acc')
plt.show()